In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler
import tensorflow as tf
from pathlib import Path
from time import strftime

In [33]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs
print('Tensorboard launched at localhost:6006')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

Tensorboard launched at localhost:6006


In [34]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [35]:
def get_run_dir(root='my_logs'):
    return Path(root) / strftime('run_%Y_%m_%d_%H_%M_%S')

In [36]:
def get_score(true_values, predicted_values):
    mae_ = mean_absolute_error(true_values, predicted_values)
    mse_ = mean_squared_error(true_values, predicted_values)
    r2_ = r2_score(true_values, predicted_values)

    return mae_, mse_, r2_

# Loading data

In [37]:
dataset = pd.read_csv('dataset.csv')

In [38]:
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :-3]
labels = dataset.iloc[:, -2:]

In [39]:
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2, random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full, test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

# Scaling personality values to range 0 - 1

In [40]:
mms = MinMaxScaler()

data_train_mms = mms.fit_transform(data_train)
data_train_mms = pd.DataFrame(data_train_mms, columns=data_train.columns)
data_validation_mms = mms.fit_transform(data_validation)
data_validation_mms = pd.DataFrame(data_validation_mms, columns=data_validation.columns)

# Scaling input values to Gaussian distributions

In [41]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train_mms)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train_mms.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation_mms)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation_mms.columns)

# Training

In [42]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
hidden1 = tf.keras.layers.Dense(64, activation='relu')(normalized)
hidden2 = tf.keras.layers.Dense(128, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(64, activation='relu')(hidden2)
output = tf.keras.layers.Dense(1)(hidden3)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 13)]              0         
                                                                 
 normalization_4 (Normalizat  (None, 13)               27        
 ion)                                                            
                                                                 
 dense_10 (Dense)            (None, 64)                896       
                                                                 
 dense_11 (Dense)            (None, 128)               8320      
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                           

In [43]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              metrics=['mae'])

In [44]:
norm_layer.adapt(data_train_qt)

In [45]:
n_epochs = 100

run_dir = get_run_dir()
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir, profile_batch=(0, n_epochs))

history = model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation), callbacks=[tensorboard_cb])

Epoch 1/100
570/570 [==============================] - 3s 4ms/step - loss: 5.9985 - mae: 2.0375 - val_loss: 6.8276 - val_mae: 2.1228
Epoch 2/100
570/570 [==============================] - 2s 3ms/step - loss: 4.4992 - mae: 1.8099 - val_loss: 6.6276 - val_mae: 2.0985
Epoch 3/100
570/570 [==============================] - 2s 4ms/step - loss: 4.4112 - mae: 1.7900 - val_loss: 6.5176 - val_mae: 2.0856
Epoch 4/100
570/570 [==============================] - 3s 5ms/step - loss: 4.3825 - mae: 1.7826 - val_loss: 6.6022 - val_mae: 2.0927
Epoch 5/100
570/570 [==============================] - 4s 6ms/step - loss: 4.3674 - mae: 1.7780 - val_loss: 6.5228 - val_mae: 2.0848
Epoch 6/100
570/570 [==============================] - 3s 5ms/step - loss: 4.3571 - mae: 1.7750 - val_loss: 6.3713 - val_mae: 2.0731
Epoch 7/100
570/570 [==============================] - 2s 4ms/step - loss: 4.3500 - mae: 1.7725 - val_loss: 6.4282 - val_mae: 2.0765
Epoch 8/100
570/570 [==============================] - 2s 4ms/step - 

In [46]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation.iloc[:, 1], predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

215/215 [==============================] - 1s 2ms/step

Values for validation set:
MAE: 1.6815580405980222
MSE: 4.345557510751711
RMSE: 2.0846000841292582
R2:  -0.09237007209762127


# Overfitting?

In [47]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
output = tf.keras.layers.Dense(1)(normalized)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 13)]              0         
                                                                 
 normalization_5 (Normalizat  (None, 13)               27        
 ion)                                                            
                                                                 
 dense_14 (Dense)            (None, 1)                 14        
                                                                 
Total params: 41
Trainable params: 14
Non-trainable params: 27
_________________________________________________________________


In [48]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])

In [49]:
norm_layer.adapt(data_train_qt)

In [50]:
n_epochs = 50

# run_dir = get_run_dir()
# tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir, profile_batch=(0, n_epochs))

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/50
570/570 [==============================] - 3s 4ms/step - loss: 30.2238 - mae: 5.0168 - val_loss: 51.8970 - val_mae: 6.7401
Epoch 2/50
570/570 [==============================] - 2s 4ms/step - loss: 24.6044 - mae: 4.4740 - val_loss: 31.2139 - val_mae: 4.9989
Epoch 3/50
570/570 [==============================] - 2s 4ms/step - loss: 20.1203 - mae: 3.9672 - val_loss: 22.5924 - val_mae: 4.0990
Epoch 4/50
570/570 [==============================] - 2s 4ms/step - loss: 16.4173 - mae: 3.5159 - val_loss: 16.6600 - val_mae: 3.4828
Epoch 5/50
570/570 [==============================] - 2s 4ms/step - loss: 13.3575 - mae: 3.1311 - val_loss: 13.0246 - val_mae: 3.0779
Epoch 6/50
570/570 [==============================] - 2s 4ms/step - loss: 10.8619 - mae: 2.8080 - val_loss: 10.8820 - val_mae: 2.8240
Epoch 7/50
570/570 [==============================] - 2s 4ms/step - loss: 8.8696 - mae: 2.5412 - val_loss: 8.9322 - val_mae: 2.5722
Epoch 8/50
570/570 [==============================] - 2s 4ms/ste

In [51]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation.iloc[:, 1], predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

215/215 [==============================] - 1s 2ms/step

Values for validation set:
MAE: 1.6824115542544424
MSE: 4.349000936893388
RMSE: 2.085425840660221
R2:  -0.093235667744058


# Check if data from all subjects works better

In [52]:
dataset = pd.read_csv('dataset_full.csv')
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :-3]
labels = dataset.iloc[:, -2:]
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2,
                                                                              random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full,
                                                                                test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

In [53]:
mms = MinMaxScaler()

data_train_mms = mms.fit_transform(data_train)
data_train_mms = pd.DataFrame(data_train_mms, columns=data_train.columns)
data_validation_mms = mms.fit_transform(data_validation)
data_validation_mms = pd.DataFrame(data_validation_mms, columns=data_validation.columns)

In [54]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train_mms)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train_mms.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation_mms)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation_mms.columns)

In [55]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
output = tf.keras.layers.Dense(1)(normalized)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 13)]              0         
                                                                 
 normalization_6 (Normalizat  (None, 13)               27        
 ion)                                                            
                                                                 
 dense_15 (Dense)            (None, 1)                 14        
                                                                 
Total params: 41
Trainable params: 14
Non-trainable params: 27
_________________________________________________________________


In [56]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])

In [57]:
norm_layer.adapt(data_train_qt)

In [58]:
n_epochs = 50

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/50
2394/2394 [==============================] - 7s 3ms/step - loss: 22.4080 - mae: 4.1942 - val_loss: 15.7213 - val_mae: 3.3813
Epoch 2/50
2394/2394 [==============================] - 8s 3ms/step - loss: 9.5326 - mae: 2.6213 - val_loss: 7.3244 - val_mae: 2.3336
Epoch 3/50
2394/2394 [==============================] - 7s 3ms/step - loss: 5.0254 - mae: 1.9477 - val_loss: 6.2214 - val_mae: 2.0966
Epoch 4/50
2394/2394 [==============================] - 8s 3ms/step - loss: 4.3868 - mae: 1.7931 - val_loss: 6.1987 - val_mae: 2.0759
Epoch 5/50
2394/2394 [==============================] - 8s 3ms/step - loss: 4.3777 - mae: 1.7825 - val_loss: 6.2055 - val_mae: 2.0744
Epoch 6/50
2394/2394 [==============================] - 10s 4ms/step - loss: 4.3779 - mae: 1.7831 - val_loss: 6.2063 - val_mae: 2.0749
Epoch 7/50
2394/2394 [==============================] - 10s 4ms/step - loss: 4.3772 - mae: 1.7823 - val_loss: 6.2410 - val_mae: 2.0723
Epoch 8/50
2394/2394 [==============================] - 11

In [59]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation.iloc[:, 1], predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

895/895 [==============================] - 1s 2ms/step

Values for validation set:
MAE: 1.7037912217032545
MSE: 4.461603027688486
RMSE: 2.112250701902709
R2:  -0.0961470428518092


Trying big model again

In [60]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
hidden1 = tf.keras.layers.Dense(64, activation='relu')(normalized)
hidden2 = tf.keras.layers.Dense(128, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(64, activation='relu')(hidden2)
output = tf.keras.layers.Dense(1)(hidden3)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 13)]              0         
                                                                 
 normalization_7 (Normalizat  (None, 13)               27        
 ion)                                                            
                                                                 
 dense_16 (Dense)            (None, 64)                896       
                                                                 
 dense_17 (Dense)            (None, 128)               8320      
                                                                 
 dense_18 (Dense)            (None, 64)                8256      
                                                                 
 dense_19 (Dense)            (None, 1)                 65        
                                                           

In [61]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])
norm_layer.adapt(data_train_qt)

In [62]:
n_epochs = 25

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/25
2394/2394 [==============================] - 14s 6ms/step - loss: 4.5915 - mae: 1.8098 - val_loss: 6.4162 - val_mae: 2.1450
Epoch 2/25
2394/2394 [==============================] - 8s 4ms/step - loss: 4.3485 - mae: 1.7673 - val_loss: 6.9066 - val_mae: 2.2557
Epoch 3/25
2394/2394 [==============================] - 9s 4ms/step - loss: 4.3270 - mae: 1.7618 - val_loss: 7.6310 - val_mae: 2.3755
Epoch 4/25
2394/2394 [==============================] - 8s 3ms/step - loss: 4.3184 - mae: 1.7594 - val_loss: 6.9558 - val_mae: 2.2637
Epoch 5/25
2394/2394 [==============================] - 8s 3ms/step - loss: 4.3048 - mae: 1.7563 - val_loss: 8.7220 - val_mae: 2.5412
Epoch 6/25
2394/2394 [==============================] - 8s 3ms/step - loss: 4.2981 - mae: 1.7547 - val_loss: 10.1886 - val_mae: 2.7359
Epoch 7/25
2394/2394 [==============================] - 9s 4ms/step - loss: 4.2902 - mae: 1.7529 - val_loss: 9.2388 - val_mae: 2.6119
Epoch 8/25
2394/2394 [==============================] - 8s 3

In [63]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation.iloc[:, 1], predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

895/895 [==============================] - 2s 2ms/step

Values for validation set:
MAE: 1.6253704886307228
MSE: 4.150014941293899
RMSE: 2.037158545939392
R2:  -0.019594656328476567
